In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the base model architecture
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(38, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set the learning rate and batch size for incremental learning
learning_rate = 0.001
batch_size = 32

# Load the initial training and testing data
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(r"C:\Users\lenovo\Downloads\New plant disease dataset\train",
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')
test_generator = test_datagen.flow_from_directory(r"C:\Users\lenovo\Downloads\New plant disease dataset\test",
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode="categorical"
    )

# Train the initial model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Initialize the model weights
model_weights = model.get_weights()
# Loop over new training data and update the model weights incrementally
for epoch in range(10, 20):
    # Get a new batch of training data
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    trai
n_generator = train_datagen.flow_from_directory(r"C:\Users\lenovo\Downloads\New plant disease dataset\train",
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')
    
    # Compute the gradients using the new batch of training data
    with tf.GradientTape() as tape:
        # Set the model weights to the current state
        model.set_weights(model_weights)
        
        # Compute the loss and gradients on the new batch of training data
        x_batch, y_batch = train_generator.next()
        y_pred = model(x_batch)
        loss = tf.keras.losses.categorical_crossentropy(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
        
    # Update the model weights using SGD
    for i, grad in enumerate(grads):
        model_weights[i] -= learning_rate * grad.numpy()
    
    # Save the updated model weights
    model.set_weights(model_weights)
    
    # Evaluate the model on the testing set
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(r"C:\Users\lenovo\Downloads\New plant disease dataset\test",
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode="categorical")
       
    test_loss, test_acc = model.evaluate(test_generator)
    
    print('Epoch: {}, Test Loss: {}, Test Accuracy: {}'.format(epoch, test_loss, test_acc))

Found 54956 images belonging to 38 classes.
Found 33 images belonging to 1 classes.
Epoch 1/10
1718/1718 [==============================] - 33214s 19s/step - loss: 0.8942 - accuracy: 0.7354 - val_loss: 570.3740 - val_accuracy: 0.0303
Epoch 2/10
1082/1718 [=================>............] - ETA: 5:22:53 - loss: 0.2812 - accuracy: 0.9115

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy curves
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Plot loss curves
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
